In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
from pandas.core.common import SettingWithCopyWarning
import plotly.graph_objects as go
import plotly.express as px
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# Load in price data
price_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/prices')

In [3]:
price_data.id = price_data.id.astype('str')

In [4]:
miami_example = price_data[price_data['id'] == '52364652.0']

In [5]:
from datetime import datetime

miami_example['pull_time'] = pd.to_datetime(miami_example['pull_time'])
miami_example['pull_time'] = miami_example['pull_time'].dt.date


In [6]:
miami_example_by_pull_time = miami_example.groupby(['id','pull_time'])['cleaning_fee','service_fee','total_price'].mean().reset_index()
miami_example_by_pull_time_count = miami_example.groupby(['pull_time','check_in'])['cleaning_fee','service_fee','total_price'].count().reset_index()

In [8]:
# Create traces
miami_example_grouped_fig = go.Figure()
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['service_fee'],
                    mode='lines',
                    name='Service Fee'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['total_price'],
                    mode='lines', 
                    name='Total Price'))
miami_example_grouped_fig.update_layout(title_text="Pricing Data by Date", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/price_data_line_fig_miami_example.png"
miami_example_grouped_fig.write_image(filename, engine='kaleido', width=2000, height=500)
miami_example_grouped_fig.show()

In [11]:
import plotly.express as px
fig = px.histogram(miami_example_by_pull_time_count, x="pull_time",nbins=50)
fig.show()

In [62]:
# Reference https://plotly.com/python/histograms/ under "Accessing the counts (y-axis) values"

df = px.data.tips()
# create the bins
counts, bins = np.histogram(.total_bill, bins=range(0, 60, 5))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'total_bill', 'y':'count'})
fig.show()

,id,pull_time,cleaning_fee,service_fee,total_price
0,52364652.0,2022-01-05,89,89,89
1,52364652.0,2022-02-05,89,89,89
2,52364652.0,2022-03-05,89,89,89
3,52364652.0,2022-04-05,89,89,89
4,52364652.0,2022-04-18,9,9,9
5,52364652.0,2022-04-19,98,98,98
6,52364652.0,2022-04-20,89,89,89
7,52364652.0,2022-04-21,89,89,89
8,52364652.0,2022-04-22,89,89,89
9,52364652.0,2022-04-23,89,89,89


In [12]:
# aggregate price data by mean and std grouped by id

agg_price_data = price_data.groupby(['id']).agg({'cleaning_fee' :['mean','std'], 'service_fee' :['mean','std'], 'total_price' :['mean','std']}).reset_index()

In [15]:
agg_price_data

id cleaning_fee      service_fee              total_price  \
                         mean  std        mean         std         mean   
0     10121456.0        488.0  0.0  421.258880  130.816750  2495.932992   
1     10123207.0          NaN  NaN   67.158714   27.476552   475.509645   
2     10142320.0        100.0  0.0   67.900592    2.999240   379.337278   
3     10162954.0        250.0  0.0  204.432859   49.952842  1197.032320   
4     10192685.0        215.0  0.0  245.826278   84.961106  1525.604470   
...          ...          ...  ...         ...         ...          ...   
5601   9919384.0         99.0  0.0   37.710231    2.709398   167.541691   
5602   9928289.0          NaN  NaN  126.902066   27.965814   898.868182   
5603   9940136.0        100.0  0.0   52.000000    0.000000   266.000000   
5604    994967.0        150.0  0.0   39.000000    0.000000   125.000000   
5605   9952412.0          5.0  0.0    7.882532    1.099555    50.843940   

                  
             std  
0     927.244837  
1     194.277664  
2      19.994934  
3     353.963683  
4     602.447862  
...          ...  
5601   19.868917  
5602  196.850681  
5603    0.000000  
5604    0.000000  
5605    7.961069  

[5606 rows x 7 columns]

In [ ]:
# Do a delta from month of June price vs the month of May price for all properties to see increase vs decrease Month over Month
# Then, merge each id with its zipcode, state, and city (if city is there) from the flat csv files used in the eda_summary (listings_w_zips.csv)
# Then, groupby on zipcode with mean on delta from June-May to get average price increase vs decrease in that state/zip/city